### classify newsgroup docsby considering each document a simple mean of its individual word embeddings, using one-vs-all SVM

In [1]:
import csv
import os
import re
import sys
import numpy as np
import tensorflow as tf

from IPython.display import SVG

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils.vis_utils import model_to_dot
from keras.layers import Dense, Dropout, Input, Flatten, Activation, Lambda
from keras.layers import Conv1D, MaxPooling1D, Embedding, GlobalAvgPool1D
from keras.models import Model, Sequential

from sklearn import preprocessing, svm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import ParameterGrid
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.pipeline import Pipeline

# my stuff in the helpers/ directory
from helpers import files_helper, texts_helper, metrics_helper, tags_helper

Using TensorFlow backend.


In [2]:
from keras import backend as K

In [3]:
SEED=np.random.randint(1,1000)
SEED

203

In [4]:
np.random.seed(seed=SEED)

In [5]:
texts,labels_index,labels_0 = files_helper.read_20_newgroup_files()       

In [6]:
preprocessed_texts = texts

In [7]:
MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH = 1000
VALIDATION_SPLIT = 0.2
EMBEDDING_DIM=100
LABELS_MIN_DOC_COUNT = int(10)
BATCH_SIZE=32
NUM_EPOCHS=10
TOKENIZER_FILTERS='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n' # default keras filters

In [8]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS,
                     filters=TOKENIZER_FILTERS)
tokenizer.fit_on_texts(preprocessed_texts)
sequences = tokenizer.texts_to_sequences(preprocessed_texts)
word_index = tokenizer.word_index
inverse_word_index = texts_helper.build_inverse_word_index(word_index)

In [9]:
[inverse_word_index[idx] for idx in sequences[0]]

['archive',
 'name',
 'atheism',
 'resources',
 'alt',
 'atheism',
 'archive',
 'name',
 'resources',
 'last',
 'modified',
 '11',
 'december',
 '1992',
 'version',
 '1',
 '0',
 'atheist',
 'resources',
 'addresses',
 'of',
 'atheist',
 'organizations',
 'usa',
 'freedom',
 'from',
 'religion',
 'foundation',
 'darwin',
 'fish',
 'bumper',
 'stickers',
 'and',
 'assorted',
 'other',
 'atheist',
 'are',
 'available',
 'from',
 'the',
 'freedom',
 'from',
 'religion',
 'foundation',
 'in',
 'the',
 'us',
 'write',
 'to',
 'p',
 'o',
 'box',
 '750',
 'madison',
 'wi',
 'telephone',
 '608',
 '256',
 '8900',
 'evolution',
 'designs',
 'evolution',
 'designs',
 'sell',
 'the',
 'darwin',
 'fish',
 "it's",
 'a',
 'fish',
 'symbol',
 'like',
 'the',
 'ones',
 'christians',
 'stick',
 'on',
 'their',
 'cars',
 'but',
 'with',
 'feet',
 'and',
 'the',
 'word',
 'darwin',
 'written',
 'inside',
 'the',
 'deluxe',
 '3d',
 'plastic',
 'fish',
 'is',
 '4',
 '95',
 'in',
 'the',
 'us',
 'write',
 'to

In [10]:
# word => embedding
embeddings_index = files_helper.read_glove_wiki(d=EMBEDDING_DIM)

In [11]:
data = pad_sequences(sequences,maxlen=MAX_SEQUENCE_LENGTH)

labels_1 = to_categorical(np.asarray(labels_0))

indices = np.arange(data.shape[0])
np.random.shuffle(indices)

data = data[indices]
labels_2 = labels_1[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

X_train = data[:-num_validation_samples]
Y_train = labels_2[:-num_validation_samples]
X_val = data[-num_validation_samples:]
Y_val = labels_2[-num_validation_samples:]

data.shape,labels_2.shape

((19997, 1000), (19997, 20))

In [12]:
embedding_matrix = np.zeros((len(word_index)+1,EMBEDDING_DIM))

for word,i in word_index.items():
    
    if i >= MAX_NB_WORDS:
        continue
    
    embedding_vector = embeddings_index.get(word)
    
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        
        
embedding_matrix.shape        

(174075, 100)

In [13]:
num_labels = labels_1.shape[1]

model = Sequential()

model.add(Embedding(len(word_index)+1,
                           EMBEDDING_DIM,
                           weights=[embedding_matrix],
                           input_length=MAX_SEQUENCE_LENGTH,
                           trainable = False))

# no need for extra layers if we aren't training the pooling layers
model.add(GlobalAvgPool1D(name='avg_pooling'))
model.add(Dense(64,activation='relu',name='dense_1'))
model.add(Dropout(0.5,name="dropout_1"))
model.add(Dense(64,activation='relu',name="dense_2"))
model.add(Dropout(0.5,name="dropout_2"))
model.add(Dense(num_labels, activation='softmax',name="softmax_output"))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [14]:
%%time

model.fit(X_train,Y_train, validation_data=(X_val, Y_val),
         epochs=NUM_EPOCHS, batch_size=BATCH_SIZE)

Train on 15998 samples, validate on 3999 samples
Epoch 1/10
15998/15998 [==============================] - 2s - loss: 2.9149 - acc: 0.0848 - val_loss: 2.6963 - val_acc: 0.1533
Epoch 2/10
15998/15998 [==============================] - 1s - loss: 2.5780 - acc: 0.1553 - val_loss: 2.3269 - val_acc: 0.2478
Epoch 3/10
15998/15998 [==============================] - 2s - loss: 2.3598 - acc: 0.1951 - val_loss: 2.1511 - val_acc: 0.2918
Epoch 4/10
15998/15998 [==============================] - 2s - loss: 2.2370 - acc: 0.2327 - val_loss: 2.0326 - val_acc: 0.3321
Epoch 5/10
15998/15998 [==============================] - 2s - loss: 2.1548 - acc: 0.2593 - val_loss: 1.9642 - val_acc: 0.3568
Epoch 6/10
15998/15998 [==============================] - 2s - loss: 2.0888 - acc: 0.2786 - val_loss: 1.8866 - val_acc: 0.3731
Epoch 7/10
15998/15998 [==============================] - 2s - loss: 2.0481 - acc: 0.2923 - val_loss: 1.8373 - val_acc: 0.3883
Epoch 8/10
15998/15998 [==============================] - 2s -

In [15]:
get_pooling_layer_output = K.function([model.layers[0].input, K.learning_phase()],
                                  [model.layers[1].output])

In [16]:
new_X_train = []
for raw_doc_features in X_train:
    layer_output = get_pooling_layer_output([raw_doc_features.reshape(1,-1),0])
    new_X_train.append(layer_output[0].ravel())
new_X_train = np.vstack(new_X_train)

new_X_train.shape

(15998, 100)

In [18]:
new_X_val = []
for raw_doc_features in X_val:
    layer_output = get_pooling_layer_output([raw_doc_features.reshape(1,-1),0])
    new_X_val.append(layer_output[0].ravel())
new_X_val = np.vstack([new_X_val])

new_X_val.shape

(3999, 100)

In [21]:
clf = OneVsRestClassifier(Pipeline([
    ('clf', svm.LinearSVC()),
]),n_jobs=-1)
parameters = [
    { 
          "estimator__clf__penalty": ["l1"],
          "estimator__clf__dual":[False],
          "estimator__clf__multi_class":["ovr"],
          "estimator__clf__tol": [0.001],

    }
]

In [22]:
best_score = float("-inf")

for g in ParameterGrid(parameters):
    clf.set_params(**g)
    %time clf.fit(new_X_train,Y_train)
    
    Y_pred_train = clf.predict(new_X_train)    
    Y_pred_val = clf.predict(new_X_val)
    
    train_score = accuracy_score(Y_train,Y_pred_train)
    val_score = accuracy_score(Y_val,Y_pred_val)
    
    current_score = val_score
    
    print("train accuracy: {}".format(train_score))
    print("val accuracy: {}".format(val_score))
    print("grid: {}".format(g))
    print("")
    
    if current_score > best_score:
        best_score = current_score

        best_grid = g

CPU times: user 676 ms, sys: 336 ms, total: 1.01 s
Wall time: 44.7 s
train accuracy: 0.2303412926615827
val accuracy: 0.21980495123780946
grid: {'estimator__clf__penalty': 'l1', 'estimator__clf__tol': 0.001, 'estimator__clf__dual': False, 'estimator__clf__multi_class': 'ovr'}



In [29]:
# SUCKS!!!
# http://scikit-learn.org/stable/modules/svm.html#tips-on-practical-use

sc= preprocessing.StandardScaler()
new_new_X_train = sc.fit_transform(new_X_train)
new_new_X_val = sc.transform(new_X_val)

clf = OneVsRestClassifier(Pipeline([
    ('clf', svm.SVC()),
]),n_jobs=-1)

C_range = np.logspace(-2, 10, 8)
gamma_range = np.logspace(-9, 3, 8)
parameters = [{
    'estimator__clf__gamma':gamma_range,
    'estimator__clf__C':C_range
}]

best_score = float("-inf")

for g in ParameterGrid(parameters):
    clf.set_params(**g)
    %time clf.fit(new_new_X_train,Y_train)
    
    Y_pred_train = clf.predict(new_new_X_train)    
    Y_pred_val = clf.predict(new_new_X_val)
    
    train_score = accuracy_score(Y_train,Y_pred_train)
    val_score = accuracy_score(Y_val,Y_pred_val)
    
    current_score = val_score
    
    print("train accuracy: {}".format(train_score))
    print("val accuracy: {}".format(val_score))
    print("grid: {}".format(g))
    print("")
    
    if current_score > best_score:
        best_score = current_score
        best_grid = g

CPU times: user 544 ms, sys: 440 ms, total: 984 ms
Wall time: 23 s
train accuracy: 0.0
val accuracy: 0.0
grid: {'estimator__clf__gamma': 1.0000000000000001e-09, 'estimator__clf__C': 0.01}

CPU times: user 580 ms, sys: 308 ms, total: 888 ms
Wall time: 23 s
train accuracy: 0.0
val accuracy: 0.0
grid: {'estimator__clf__gamma': 5.1794746792312124e-08, 'estimator__clf__C': 0.01}

CPU times: user 552 ms, sys: 308 ms, total: 860 ms
Wall time: 22.4 s
train accuracy: 0.0
val accuracy: 0.0
grid: {'estimator__clf__gamma': 2.6826957952797274e-06, 'estimator__clf__C': 0.01}

CPU times: user 532 ms, sys: 356 ms, total: 888 ms
Wall time: 23.1 s
train accuracy: 0.0
val accuracy: 0.0
grid: {'estimator__clf__gamma': 0.0001389495494373136, 'estimator__clf__C': 0.01}

CPU times: user 608 ms, sys: 332 ms, total: 940 ms
Wall time: 26.9 s
train accuracy: 0.0
val accuracy: 0.0
grid: {'estimator__clf__gamma': 0.0071968567300115137, 'estimator__clf__C': 0.01}

CPU times: user 1.1 s, sys: 740 ms, total: 1.84 s
W

train accuracy: 0.9579947493436679
val accuracy: 0.24956239059764943
grid: {'estimator__clf__gamma': 0.37275937203149379, 'estimator__clf__C': 71968.567300115144}

CPU times: user 4.34 s, sys: 2.52 s, total: 6.86 s
Wall time: 16min 55s
train accuracy: 0.9544943117889736
val accuracy: 0.011002750687671918
grid: {'estimator__clf__gamma': 19.306977288832456, 'estimator__clf__C': 71968.567300115144}

CPU times: user 4.25 s, sys: 2.43 s, total: 6.68 s
Wall time: 16min 29s


KeyboardInterrupt: 